# Segment Data from the Calcbench API

In [ ]:
%pip install calcbench-api-client qgrid

In [ ]:
import calcbench as cb
import pandas as pd
import qgrid

In [ ]:
tickers = cb.tickers(index="DJIA")

In [ ]:
operating_segment_revenue = cb.dimensional(
    company_identifiers=tickers,
    metrics=["OperatingSegmentRevenue", "OperatingSegmentOperatingIncome"],
    period_type="annual",
)

In [ ]:
qgrid.show_grid(operating_segment_revenue)